<a href="https://colab.research.google.com/github/hmaldonado12/7506R-1C2024-GRUPO6/blob/main/tp2/Ensamble_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
import re

In [27]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv/train.csv')
train_df.head()

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo


In [29]:
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv/test.csv')
test_df.head()

,ID,review_es
0,60000,La mayor virtud de esta película es su existen...
1,60001,"No soy un experto cinéfilo, pero pocas veces m..."
2,60002,Si no eres un incondicional del humor estilo T...
3,60003,"No sé qué está pasando, si la gente se deja ll..."
4,60004,"Pero cuando amanece,y me quedo solo,siento en ..."


In [30]:
# ----- PARAMETROS DEL OUTPUT -----
# MAX_ROWS: cantidad limite a mostrar, si se excede se resume (...)
# MIN_ROWS: cantidad de filas a mostrar pasado el corte
# MAX_COLWIDTH: ancho de columnas por cada output, si se excede se resume(...)

pd.reset_option("display.min_rows")
pd.reset_option("display.max_colwidth")
# pd.options.display.min_rows = 100000
# pd.options.display.max_colwidth = 1000

In [23]:
# DEBUG: almaceno cada una de las palabras
# global_tokens = []

# Creo copias del dataset para procesar
token_train_df = train_df.copy()
token_test_df = test_df.copy()

def preprocess_text(text):
    # Llevo texto a minúsculas
    text = text.lower()

    # Expresión regular para eliminar caracteres '¿', '¡', y '`'
    text = re.sub(r'[¡¿\`\']', ' ', text)

    # Expresión regular para separar palabras con puntos intermedios
    text = re.sub(r'([\wáéíóúü]+)\.+([\wáéíóúü]+)', r'\1 \2', text)

    # Tokenizacion
    tokens = word_tokenize(text, language='spanish')

    # Elimino signos de puntuación
    tokens = [word for word in tokens if word.isalpha()]

    # global_tokens.extend(tokens) # DEBUG
    return " ".join(tokens)

token_train_df['review_es'] = train_df['review_es'].apply(preprocess_text)
token_test_df['review_es'] = test_df['review_es'].apply(preprocess_text)

In [31]:
# Cargo las stopwords del corpus de NLTK
stoplist = stopwords.words('spanish')
stoplist_eng = stopwords.words('english')

# Tokenizo el stoplist con el mismo tokenizador del corpus principal
stoplist_tokenized = []
for w in (stoplist + stoplist_eng):
    stoplist_tokenized = stoplist_tokenized + word_tokenize(w)

# Armo una lista sin repeticiones
stoplist_tokenized = list(set(stoplist_tokenized))

# stoplist_tokenized
# len(stoplist + stoplist_eng)
# len(stoplist_tokenized)

In [24]:

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_df['review_es'])
y = train_df['sentimiento'].map({'positivo': 1, 'negativo': 0})

# Dividir los datos en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)


MultinomialNB()

In [33]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [34]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [35]:
def create_nn_model():
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

nn_model = KerasClassifier(build_fn=create_nn_model, epochs=10, batch_size=32, verbose=1)
nn_model._estimator_type = "classifier"
nn_model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 446s 355ms/step - accuracy: 0.8317 - loss: 0.3897
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 508s 360ms/step - accuracy: 0.9631 - loss: 0.0994
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 506s 363ms/step - accuracy: 0.9889 - loss: 0.0349
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 505s 366ms/step - accuracy: 0.9937 - loss: 0.0190
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 456s 365ms/step - accuracy: 0.9960 - loss: 0.0132
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 498s 361ms/step - accuracy: 0.9980 - loss: 0.0069
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 506s 364ms/step - accuracy: 0.9985 - loss: 0.0051
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 501s 363ms/step - accuracy: 0.9981 - loss: 0.0066
Epoch 9/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 500s 361ms/step - accuracy: 0.9985 - loss: 0.0059
Epoch 10/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 516s 372ms/step - accuracy: 0.9984 - loss: 0.0050


KerasClassifier(
	model=None
	build_fn=<function create_nn_model at 0x7eee74d95240>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	class_weight=None
)

In [36]:
ensemble_model = VotingClassifier(estimators=[
    ('nb', nb_model),
    ('rf', rf_model),
    ('xgb', xgb_model),
    ('nn', nn_model)
], voting='soft')

ensemble_model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 457s 364ms/step - accuracy: 0.8318 - loss: 0.3883
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 463s 371ms/step - accuracy: 0.9610 - loss: 0.1021
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 451s 360ms/step - accuracy: 0.9870 - loss: 0.0359
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 505s 363ms/step - accuracy: 0.9950 - loss: 0.0162
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 458s 366ms/step - accuracy: 0.9960 - loss: 0.0118
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 508s 371ms/step - accuracy: 0.9973 - loss: 0.0087
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 504s 372ms/step - accuracy: 0.9985 - loss: 0.0054
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 463s 371ms/step - accuracy: 0.9992 - loss: 0.0028
Epoch 9/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 463s 370ms/step - accuracy: 0.9984 - loss: 0.0050
Epoch 10/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 495s 365ms/step - accuracy: 0.9988 - loss: 0.0088


VotingClassifier(estimators=[('nb', MultinomialNB()),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, gamma=None,
                                            gro...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('nn',
                              KerasClassifier(batch_size=32, build_fn=<function create_nn_model at 0x7eee74d95240>, epochs=10))],
                 voting='soft')

In [37]:
y_pred = ensemble_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy of the ensemble model: {accuracy}')


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Accuracy of the ensemble model: 0.8897


In [41]:
X_test = vectorizer.transform(test_df['review_es'])
test_predictions = ensemble_model.predict(X_test)



269/269 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step


In [49]:
salida = pd.DataFrame({'ID': test_df['ID'], 'sentimiento': test_predictions})
salida['sentimiento'] = salida['sentimiento'].map({1: 'positivo', 0: 'negativo'})

# Guardar el archivo para subir a Kaggle
salida.to_csv('/content/drive/My Drive/Colab Notebooks/Ensamble.csv', index=False)

In [50]:
salida

,ID,sentimiento
0,60000,negativo
1,60001,negativo
2,60002,negativo
3,60003,negativo
4,60004,negativo
...,...,...
8594,68594,positivo
8595,68595,negativo
8596,68596,positivo
8597,68597,negativo
